In [1]:
from bs4 import BeautifulSoup
import requests



In [2]:
page = requests.get("https://forecast.weather.gov/MapClick.php?lat=39.74&lon=-104.992")
soup = BeautifulSoup(page.content, 'html.parser')

In [3]:
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")
tonight = forecast_items[0]
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Tonight
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Tonight: Partly cloudy, with a low around 39. South southwest wind 3 to 5 mph. " class="forecast-icon" src="newimages/medium/nsct.png" title="Tonight: Partly cloudy, with a low around 39. South southwest wind 3 to 5 mph. "/>
 </p>
 <p class="short-desc">
  Partly Cloudy
 </p>
 <p class="temp temp-low">
  Low: 39 °F
 </p>
</div>


In [4]:
period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp").get_text()
print(period)
print(short_desc)
print(temp)

Tonight
Partly Cloudy
Low: 39 °F


In [5]:
img = tonight.find("img")
desc = img['title']
print(desc)

Tonight: Partly cloudy, with a low around 39. South southwest wind 3 to 5 mph. 


In [6]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['Tonight',
 'Tuesday',
 'TuesdayNight',
 'Wednesday',
 'WednesdayNight',
 'Thursday',
 'ThursdayNight',
 'Friday',
 'FridayNight']

In [9]:
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]
print(short_descs)
print(temps)
print(descs)

['Partly Cloudy', 'Mostly Sunny', 'Mostly Cloudy', 'Rain Likely', 'Snow Likelythen ChanceSnow', 'Chance Snow', 'Chance Snow', 'Mostly Sunny', 'Partly Cloudy']
['Low: 39 °F', 'High: 71 °F', 'Low: 39 °F', 'High: 51 °F', 'Low: 26 °F', 'High: 37 °F', 'Low: 23 °F', 'High: 48 °F', 'Low: 26 °F']
['Tonight: Partly cloudy, with a low around 39. South southwest wind 3 to 5 mph. ', 'Tuesday: Mostly sunny, with a high near 71. Light and variable wind becoming south southwest around 5 mph. ', 'Tuesday Night: Mostly cloudy, with a low around 39. Calm wind becoming west southwest around 5 mph after midnight. ', 'Wednesday: Rain likely, mainly after 11am.  Cloudy, with a high near 51. Calm wind becoming north 5 to 8 mph in the morning. Winds could gust as high as 15 mph.  Chance of precipitation is 70%.', 'Wednesday Night: Snow likely, mainly before 11pm.  Cloudy, with a low around 26. North wind 5 to 7 mph becoming calm  in the evening.  Chance of precipitation is 70%.', 'Thursday: A 50 percent chanc

In [10]:
import pandas as pd
weather = pd.DataFrame({
    "period": periods,
    "short_desc": short_descs,
    "temp": temps,
    "desc":descs
})
weather

,period,short_desc,temp,desc
0,Tonight,Partly Cloudy,Low: 39 °F,"Tonight: Partly cloudy, with a low around 39. ..."
1,Tuesday,Mostly Sunny,High: 71 °F,"Tuesday: Mostly sunny, with a high near 71. Li..."
2,TuesdayNight,Mostly Cloudy,Low: 39 °F,"Tuesday Night: Mostly cloudy, with a low aroun..."
3,Wednesday,Rain Likely,High: 51 °F,"Wednesday: Rain likely, mainly after 11am. Cl..."
4,WednesdayNight,Snow Likelythen ChanceSnow,Low: 26 °F,"Wednesday Night: Snow likely, mainly before 11..."
5,Thursday,Chance Snow,High: 37 °F,Thursday: A 50 percent chance of snow. Cloudy...
6,ThursdayNight,Chance Snow,Low: 23 °F,"Thursday Night: A 30 percent chance of snow, m..."
7,Friday,Mostly Sunny,High: 48 °F,"Friday: Mostly sunny, with a high near 48."
8,FridayNight,Partly Cloudy,Low: 26 °F,"Friday Night: Partly cloudy, with a low around..."


In [11]:
temp_nums = weather["temp"].str.extract("(?P<temp_num>\d+)", expand=False)
weather["temp_num"] = temp_nums.astype('int')
temp_nums

0    39
1    71
2    39
3    51
4    26
5    37
6    23
7    48
8    26
Name: temp_num, dtype: object

In [12]:
weather["temp_num"].mean()

40.0

In [13]:
is_night = weather["temp"].str.contains("Low")
weather["is_night"] = is_night
is_night

0     True
1    False
2     True
3    False
4     True
5    False
6     True
7    False
8     True
Name: temp, dtype: bool

In [14]:
weather[is_night]

,period,short_desc,temp,desc,temp_num,is_night
0,Tonight,Partly Cloudy,Low: 39 °F,"Tonight: Partly cloudy, with a low around 39. ...",39,True
2,TuesdayNight,Mostly Cloudy,Low: 39 °F,"Tuesday Night: Mostly cloudy, with a low aroun...",39,True
4,WednesdayNight,Snow Likelythen ChanceSnow,Low: 26 °F,"Wednesday Night: Snow likely, mainly before 11...",26,True
6,ThursdayNight,Chance Snow,Low: 23 °F,"Thursday Night: A 30 percent chance of snow, m...",23,True
8,FridayNight,Partly Cloudy,Low: 26 °F,"Friday Night: Partly cloudy, with a low around...",26,True
